In [131]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams.update({'font.size': 20, 'figure.figsize': (10, 8)}) # set font and plot size to be larger
pd.set_option('display.max_columns',2000) # show all columns in jupyter
pd.set_option('display.max_rows', 100) # display 100 at least rows

In [132]:
# , "MEDIAN_HH_INC"
colsToLoad =["INSTNM","ADM_RATE","ADM_RATE_ALL","ACTCMMID","ACTENMID","ACTMTMID","ACTWRMID","SAT_AVG","SAT_AVG_ALL","UGDS","HIGHDEG", "CONTROL", "TUITFTE", "INEXPFTE", "AVGFACSAL", "COSTT4_P",
             "COSTT4_A", "PCTFLOAN", "PCTPELL", "COMP_ORIG_YR2_RT", "UGDS_WHITE","UGDS_BLACK","UGDS_HISP","UGDS_ASIAN","UGDS_AIAN","UGDS_NHPI","UGDS_2MOR","UGDS_NRA","UGDS_UNKN","PPTUG_EF","COSTT4_A","COSTT4_P","TUITIONFEE_IN","TUITIONFEE_OUT","TUITIONFEE_PROG","TUITFTE","INEXPFTE","AVGFACSAL","PCTPELL","DEATH_YR3_RT","COMP_ORIG_YR3_RT","LOAN_DEATH_YR3_RT","LOAN_COMP_ORIG_YR3_RT","DEATH_YR4_RT","COMP_ORIG_YR4_RT","COMPL_RPY_1YR_RT","NONCOM_RPY_1YR_RT","AGE_ENTRY","COUNT_NWNE_P10","COUNT_WNE_P10","MN_EARN_WNE_P10","MD_EARN_WNE_P10","COMPL_RPY_1YR_RT","NONCOM_RPY_1YR_RT","COMPL_RPY_3YR_RT","NONCOM_RPY_3YR_RT","COMPL_RPY_5YR_RT","NONCOM_RPY_5YR_RT","COMPL_RPY_7YR_RT","NONCOM_RPY_7YR_RT"]
scoreCardDF = pd.read_csv("MERGED2013_14_PP.csv", index_col="INSTNM",usecols=colsToLoad)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (1606) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [133]:
def cleanPrivacySuppressed(dataFrame):
    for colName in dataFrame.columns:
        dataFrame[colName] = dataFrame[colName].apply(lambda x: np.nan if x =="PrivacySuppressed" else x)
        dataFrame[colName]= pd.to_numeric(dataFrame[colName])

In [134]:
#some columns have PrivacySuppressed as a value to indicate that the value is missing, but it is a column of type float
#let's clean that
cleanPrivacySuppressed(scoreCardDF )

In [135]:
scoreCardDF = scoreCardDF.drop(columns=[
"COMPL_RPY_3YR_RT", 
"NONCOM_RPY_3YR_RT", 
"NONCOM_RPY_5YR_RT", 
"NONCOM_RPY_1YR_RT", 
"COMPL_RPY_5YR_RT", 
"NONCOM_RPY_7YR_RT",
"COMPL_RPY_7YR_RT" ])

In [137]:
#--Simple imputer mean
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy='mean')
data_with_imputed_values = pd.DataFrame(imp.fit_transform(scoreCardDF))
#data_with_imputed_values.info()

#imp.fit(scoreCardDF)
#X = imp.transform(scoreCardDF) 
#scoreCardDF = pd.DataFrame(X, columns=scoreCardDF.columns, index=scoreCardDF.index)

In [138]:
data_with_imputed_values.columns = scoreCardDF.columns
data_with_imputed_values

,HIGHDEG,CONTROL,ADM_RATE,ADM_RATE_ALL,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SAT_AVG,SAT_AVG_ALL,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,TUITIONFEE_PROG,TUITFTE,INEXPFTE,AVGFACSAL,PCTPELL,PCTFLOAN,COMP_ORIG_YR2_RT,DEATH_YR3_RT,COMP_ORIG_YR3_RT,LOAN_DEATH_YR3_RT,LOAN_COMP_ORIG_YR3_RT,DEATH_YR4_RT,COMP_ORIG_YR4_RT,COMPL_RPY_1YR_RT,AGE_ENTRY,COUNT_NWNE_P10,COUNT_WNE_P10,MN_EARN_WNE_P10,MD_EARN_WNE_P10
0,4.0,1.0,0.898900,0.898900,17.000000,17.000000,17.000000,7.736667,823.000000,823.000000,4051.000000,0.02790,0.950100,0.008900,0.002200,0.001200,0.001000,0.000000,0.000200,0.008400,0.062200,18888.000000,18481.605977,7182.000000,12774.000000,14910.383916,9063.000000,7459.000000,7079.000000,0.711500,0.82040,0.034959,0.002784,0.097451,0.002939,0.087333,0.003384,0.241846,0.307580,20.540410,246.000000,1741.000000,34300.00000,29900.000000
1,4.0,1.0,0.867300,0.867300,25.000000,26.000000,23.000000,7.736667,1146.000000,1146.000000,11200.000000,0.59870,0.259000,0.025800,0.051800,0.002600,0.000700,0.034400,0.014000,0.013000,0.257900,19990.000000,18481.605977,7206.000000,16398.000000,14910.383916,9033.000000,17208.000000,10170.000000,0.350500,0.53970,0.150529,0.002784,0.333946,0.002939,0.332665,0.003384,0.422933,0.578436,23.458815,470.000000,3474.000000,46400.00000,40200.000000
2,4.0,2.0,0.683853,0.692173,23.125744,22.730077,22.590753,7.736667,1056.829817,1061.605091,322.000000,0.29190,0.422400,0.009300,0.003100,0.003100,0.003100,0.000000,0.000000,0.267100,0.372700,12300.000000,18481.605977,6870.000000,6870.000000,14910.383916,12057.000000,5123.000000,3849.000000,0.683900,0.76290,0.058824,0.002784,0.161850,0.002939,0.361031,0.003384,0.209581,0.385965,33.925581,57.000000,168.000000,46100.00000,40100.000000
3,4.0,1.0,0.806200,0.806200,26.000000,26.000000,25.000000,7.736667,1180.000000,1180.000000,5525.000000,0.70120,0.131000,0.033800,0.036400,0.014500,0.000200,0.016100,0.032900,0.033800,0.239500,20306.000000,18481.605977,9192.000000,21506.000000,14910.383916,8322.000000,9352.000000,9341.000000,0.328100,0.47280,0.157939,0.002784,0.284602,0.002939,0.283109,0.003384,0.393046,0.639589,24.101277,188.000000,1361.000000,50500.00000,45600.000000
4,4.0,1.0,0.512500,0.512500,17.000000,17.000000,17.000000,7.736667,830.000000,830.000000,5354.000000,0.01610,0.928500,0.011400,0.001500,0.000900,0.000700,0.006400,0.020700,0.013800,0.090200,17400.000000,18481.605977,8720.000000,15656.000000,14910.383916,7813.000000,7393.000000,6557.000000,0.826500,0.87350,0.028949,0.002784,0.115574,0.002939,0.109495,0.003384,0.187935,0.211886,20.648575,285.000000,1977.000000,29500.00000,26700.000000
5,4.0,1.0,0.565500,0.565500,26.000000,27.000000,25.000000,7.000000,1171.000000,1171.000000,28692.000000,0.78650,0.114000,0.031300,0.011200,0.004400,0.001100,0.021000,0.027800,0.002800,0.085200,26717.000000,18481.605977,9450.000000,23950.000000,14910.383916,12198.000000,9817.000000,9605.000000,0.210700,0.41480,0.086230,0.002784,0.339847,0.002939,0.335559,0.003384,0.516314,0.647702,21.527737,409.000000,3648.000000,49900.00000,42700.000000
6,2.0,1.0,0.683853,0.692173,23.125744,22.730077,22.590753,7.736667,1056.829817,1061.605091,1779.000000,0.67850,0.294500,0.011800,0.002200,0.002200,0.001100,0.000600,0.001100,0.007900,0.466000,12103.000000,18481.605977,4200.000000,7500.000000,14910.383916,2371.000000,5935.000000,5805.000000,0.651500,0.47770,0.019916,0.002784,0.037072,0.002939,0.051565,0.003384,0.020408,0.313725,24.816201,190.000000,761.000000,30900.00000,27200.000000
7,3.0,1.0,0.683853,0.692173,23.125744,22.730077,22.590753,7.736667,1056.829817,1061.605091,2999.000000,0.75130,0.106400,0.021300,0.004700,0.019000,0.000700,0.013300,0.007000,0.076400,0.550200,24355.081662,18481.605977,13857.479361,16305.035223,14910.383916,5094.000000,6176.000000,7672.000000,0.410700,0.62770,0.333333,0.002784,0.462749,0.002939,0.488918,0.003384,0.519024,0.519871,30.283219,126.000000,928.000000,4220

In [139]:
test = data_with_imputed_values["COMPL_RPY_1YR_RT"]
train = data_with_imputed_values.drop(columns=["COMPL_RPY_1YR_RT"])

In [140]:
#--split data to test and training data
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(train, test, test_size=0.2, random_state=0)

In [144]:
#--fit the Ridge model
ridgereg = Ridge(alpha=0.5,normalize=True)
ridgereg.fit(train_X, train_y)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

In [145]:
pred = ridgereg.predict(test_X)

In [146]:
ridgereg.score(test_X, test_y)

0.5489021535910275